# Scratch tbv ewcore#213: item-tag herzien

Zie ook de uitleg/voortgang in https://taiga.edwh.nl/project/remco-ewcore/us/213


## Uitlezen explain plans: 

Uitvoeren met `explain (analyze,costs, buffers, verbose) `  
Plans uitlezen: https://explain.dalibo.com/


In [59]:
%run ./__init__edwh__notebooks.ipynb

Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) 
[GCC 9.4.0]
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * postgresql://postgres@pgpool:5432/backend
Done.


## table: item-tag
 * **id**:       simpele ID compatabile met pydal. Comments moeten linken naar deze koppeling. 
 * **gid**:      globaal unieke gid voor deze relatie. NU NOG NIET GEBRUIKT
 * **item_gid**: gid datatype verwijzend naar het item , wereldwijd uniek
 * **item_id**:  unieke sleutel van het item, bedoeld voor mogelijk snellere verwerking uniek binnen deze database
 * **tag_gid**:  gid datatype verwijzend naar de tag , wereldwijd uniek
 * **tag_id**:   unieke sleutel van het item, bedoeld voor mogelijke snellere verwerking. uniek binnen deze database
 * **explanation**: een beschrijving van de relatie, indien nodig 
 * **ts_created**: wanneer aangemaakt
 * **ts_revised**: wanneer gecontroleerd of gewijzigd
 * **link_user_gid**: uuid verwijzend naar een gebruiker als laatste auteur die hier wat in gewijzigd heeft. 
 * **questionable**: boolean status die aangemerkt kan worden dat een koppeling opnieuw bekeken moet worden. Proces gegeven voor WoW van de Eddies. 

In [2]:
Tag('1274163a-be86-4395-8eef-4af66d9b81ec').name

'Omgeving Leiden'

In [3]:
TItem.gid

'fc8a7d36-e23e-4aba-8ee9-f5269ecc8dfc'

In [4]:
# %%sql 
# drop table public.item_tag; 

# create table if not exists public.item_tag
# (
#     id            serial primary key,
#     gid           uuid, 
#     item_gid      uuid,
#     item_id       integer,
#     tag_gid       uuid,
#     tag_id        integer,
#     explanation   text,
#     ts_created    timestamp,
#     ts_revised    timestamp,
#     link_user_gid uuid,
#     questionable boolean
# );

# alter table public.item_tag
#     owner to postgres;
    
# select * from item_tag; 

In [56]:
%%sql 
-- # TODO: toevoegen aan migrate 
CREATE FUNCTION fn_recursive_parent_tags(root_tag_gid text ) RETURNS TABLE (gid uuid, name text) AS $$
    with recursive tag_cte as (
    select tag.gid, tag.parents, tag.name
      from tag 
     where tag.gid = root_tag_gid
  union all -- # -------------------------------- 
    select tag.gid, tag.parents, tag.name 
      from tag inner join tag_cte on tag.gid = ANY(array_remove(regexp_split_to_array(tag_cte.parents, E'\\|'), ''))
)
    select gid::uuid, name 
     from tag_cte 
$$ LANGUAGE SQL;

 * postgresql://postgres@pgpool:5432/backend
Done.


[]

In [57]:
%%sql 
select normalize('𝔰𝔬𝔪𝔢𝔫𝔞𝔪𝔢',nfkc);

 * postgresql://postgres@pgpool:5432/backend
1 rows affected.


normalize
somename


In [7]:
%%sql 
--# explain (analyze,costs, buffers, verbose) 
select item.gid, tag.gid --,  item.name, tag.name  
from item 
    inner join tag on tag.gid = ANY(array_remove(regexp_split_to_array(tags, E'\\|'), ''))
    inner join "user" on item.author = "user".gid  and "user".email not like '%@roc.nl'
limit 10 

 * postgresql://postgres@pgpool:5432/backend
10 rows affected.


gid,gid_1
4960bc53-8f79-4272-9c42-a5572156cadd,07df9e97-2c9d-457d-bd35-65a107dbc66d
4960bc53-8f79-4272-9c42-a5572156cadd,23940a87-6f5d-4593-8ac8-ce4ce6635983
4960bc53-8f79-4272-9c42-a5572156cadd,4bac9e99-af61-44d4-8c4f-5cf7886cfeb6
4960bc53-8f79-4272-9c42-a5572156cadd,4cc54bf2-b7c8-415d-8938-7a47d9207439
4960bc53-8f79-4272-9c42-a5572156cadd,539632c3-4601-4479-8349-2ba527bbe817
4960bc53-8f79-4272-9c42-a5572156cadd,6925fc99-1dfb-47d1-99f5-6c4c3484103b
4960bc53-8f79-4272-9c42-a5572156cadd,8372c21d-c511-49ca-8d17-52d4d920c83e
4960bc53-8f79-4272-9c42-a5572156cadd,b66b968a-0801-4123-a403-ff4461652eb3
4960bc53-8f79-4272-9c42-a5572156cadd,ba3686a9-bc1e-4387-8cfc-e2737f07bc62
4960bc53-8f79-4272-9c42-a5572156cadd,ce0df9a5-8e00-46d8-8972-1a6f6032c5bf


In [8]:
# !pip install jupyterlab_myst > /dev/null 
# import jupyterlab_myst

# Materialized Views!
 * https://www.postgresql.org/docs/current/rules-materializedviews.html

:::{important}
Bij een materialized view moet je handmatig de data updaten..
```postgresql
REFRESH MATERIALIZED VIEW mv__item_tags;
REFRESH MATERIALIZED VIEW mv__tag_arrays;
``` 
::: 

## item-tags: lookup tabel item--tag
Deze koppeling gaat over de verhouding tussen items en tags, een materialized view levert tabel snelheid op een bestaande query die af en toe bijgewerkt moet worden. Dit kan vanuit Nameko met een timer. 

In [9]:
%%sql 
drop materialized view if exists mv__item_tags; 
create materialized view mv__item_tags as 
with item_tags as (
    -- produces an array of tag gids per item.gid. `array_remove` trimms the empty values due to the split
    -- and the split_to_array is similar to python's `'|a|b|c|'.split('|')` 
    select item.gid, array_remove(regexp_split_to_array(item.tags, E'\\|'), '') as tags
      from item
), used_items as (
    -- just a list of item gids 
    select item.gid, item.name 
      from item inner join "user" on item.author = "user".gid and "user".email not like '%@roc.nl'
)
select item_tags.gid::uuid as item_gid, tag.gid::uuid as tag_gid, used_items.name as item_name, tag.name as tag_name     
  from item_tags 
       inner join tag on tag.gid = ANY(item_tags.tags)
       inner join used_items on item_tags.gid = used_items.gid 

 * postgresql://postgres@pgpool:5432/backend
Done.
12444 rows affected.


[]

In [10]:
%%sql
select * from mv__item_tags 
limit 10 

 * postgresql://postgres@pgpool:5432/backend
10 rows affected.


item_gid,tag_gid,item_name,tag_name
4960bc53-8f79-4272-9c42-a5572156cadd,07df9e97-2c9d-457d-bd35-65a107dbc66d,Elke leerkracht mediawijs,digitalisering
4960bc53-8f79-4272-9c42-a5572156cadd,23940a87-6f5d-4593-8ac8-ce4ce6635983,Elke leerkracht mediawijs,ict
4960bc53-8f79-4272-9c42-a5572156cadd,4bac9e99-af61-44d4-8c4f-5cf7886cfeb6,Elke leerkracht mediawijs,competentie mediawijsheid
4960bc53-8f79-4272-9c42-a5572156cadd,4cc54bf2-b7c8-415d-8938-7a47d9207439,Elke leerkracht mediawijs,LOF (Lerarenontwikkelfonds)
4960bc53-8f79-4272-9c42-a5572156cadd,539632c3-4601-4479-8349-2ba527bbe817,Elke leerkracht mediawijs,21e eeuwse vaardigheden
4960bc53-8f79-4272-9c42-a5572156cadd,6925fc99-1dfb-47d1-99f5-6c4c3484103b,Elke leerkracht mediawijs,Met filmpje(s)
4960bc53-8f79-4272-9c42-a5572156cadd,8372c21d-c511-49ca-8d17-52d4d920c83e,Elke leerkracht mediawijs,Bestuursbrede ontwikkeling
4960bc53-8f79-4272-9c42-a5572156cadd,b66b968a-0801-4123-a403-ff4461652eb3,Elke leerkracht mediawijs,KC Talentrijk HELMOND
4960bc53-8f79-4272-9c42-a5572156cadd,ba3686a9-bc1e-4387-8cfc-e2737f07bc62,Elke leerkracht mediawijs,"Leraar, pedagogisch medewerker of coach"
4960bc53-8f79-4272-9c42-a5572156cadd,ce0df9a5-8e00-46d8-8972-1a6f6032c5bf,Elke leerkracht mediawijs,mediacoach


In [11]:
%%sql 
REFRESH MATERIALIZED VIEW mv__item_tags;

 * postgresql://postgres@pgpool:5432/backend
Done.


[]

## tag-tree - platgeslagen tree
 * https://www.postgresql.org/docs/9.1/queries-with.html
 * https://www.geeksforgeeks.org/postgresql-recursive-query-using-ctes/
 * pivot table?  https://ubiq.co/database-blog/create-pivot-table-postgresql/ 

In [12]:
%%sql 
drop materialized view if exists mv__tag_arrays; 
create materialized view mv__tag_arrays as 
select 
    tag.id, 
    tag.gid::uuid, 
    tag.name as search,
    array_remove(regexp_split_to_array(tag.children, E'\\|'), '')::uuid[] as children,
    array_remove(regexp_split_to_array(tag.parents, E'\\|'), '')::uuid[] as parents,
    array_remove(regexp_split_to_array(tag.meta_tags, E'\\|'), '')::uuid[] as meta_tags
from tag 
    


 * postgresql://postgres@pgpool:5432/backend
Done.
9006 rows affected.


[]

In [9]:
%%sql 
with recursive tag_cte as (
    select 
        null::integer as parent_id, 
        null::text as parent_gid, 
        null::text as parent_name, 
        tag.id, 
        tag.gid, 
        tag.name, 
        0 as level, 
        tag.children 
      from tag 
     where name ilike '%system%'
  union all -- # -------------------------------- 
    select 
        tag_cte.id::integer, 
        tag_cte.gid, 
        tag_cte.name,
        tag.id, 
        tag.gid, 
        tag.name, 
        level+1, 
        tag.children 
      from tag inner join tag_cte on tag.gid = ANY(array_remove(regexp_split_to_array(tag_cte.children, E'\\|'), ''))
)
select parent_id, parent_gid, parent_name, id, gid, name, level
  from tag_cte
    limit 20

 * postgresql://postgres@pgpool:5432/backend
20 rows affected.


parent_id,parent_gid,parent_name,id,gid,name,level
None,None,None,1848,19682a99-50a3-4fc0-bb67-e0f6eff5da55,System,0
1848,19682a99-50a3-4fc0-bb67-e0f6eff5da55,System,2907,801d584c-d380-4a1d-95d5-c1c85aec89da,UIRelated,1
1848,19682a99-50a3-4fc0-bb67-e0f6eff5da55,System,6544,dcaea9b5-879e-43f7-8860-67579e10166e,isUserSelectable,1
1848,19682a99-50a3-4fc0-bb67-e0f6eff5da55,System,32611,21399f87-8361-4255-8737-24245a280dc6,Omgevingen,1
1848,19682a99-50a3-4fc0-bb67-e0f6eff5da55,System,32616,572520d6-a458-4026-ad05-9b957d5712e1,GhostPlatform,1
1848,19682a99-50a3-4fc0-bb67-e0f6eff5da55,System,6595,e104e7ac-76ae-48d7-8ada-7c9fe367db8c,Regio,1
2907,801d584c-d380-4a1d-95d5-c1c85aec89da,UIRelated,5795,8b78e33d-c4ab-442e-aaf6-411fda089b03,Filter,2
2907,801d584c-d380-4a1d-95d5-c1c85aec89da,UIRelated,5803,c05e9cea-76c0-4b48-8f91-4cee4ff7f0ab,Filterbalk,2
2907,801d584c-d380-4a1d-95d5-c1c85aec89da,UIRelated,6545,67ba8cd8-b564-4cb4-b1bc-10364c5edf16,Stickers,2
2907,801d584c-d380-4a1d-95d5-c1c85aec89da,UIRelated,5807,38f7b956-6315-4676-8076-8aeef258563e,User generated tags,2


In [55]:
%%sql 
-- # Gebruik deze query om te zoeken binnen de tags en de children. 
-- # `gid` is de kolom om verder op te matchen. 
with recursive tag_cte as (
    select tag.gid, tag.parents from tag 
     where name ilike '%organiseren%'
  union -- # -------------------------------- 
    select tag.gid, tag.parents 
      from tag inner join tag_cte on tag.gid = ANY(array_remove(regexp_split_to_array(tag_cte.parents, E'\\|'), ''))
)
select gid
  from tag_cte  

 * postgresql://postgres@pgpool:5432/backend
9 rows affected.


gid
b24503d6-57b3-4e4d-87de-2d5da08e6d4b
bf2f7ada-b750-4d0b-880a-80cc27c650fc
a5729ec3-dfbb-4c90-9ec4-78662fc9f3f6
7afe35b0-881b-41bf-88ec-7e1d3fc774e3
08ee269a-76ea-49de-9ca2-f5a45680c99c
38f7b956-6315-4676-8076-8aeef258563e
c05e9cea-76c0-4b48-8f91-4cee4ff7f0ab
801d584c-d380-4a1d-95d5-c1c85aec89da
19682a99-50a3-4fc0-bb67-e0f6eff5da55


In [11]:
TSystem.walk(_print)

 System ['System']
    UIRelated ['System']
       Filter ['System']
       User generated tags ['System']
           *** hidden ***
       Filterbalk ['System', 'Filterbalk']
          Coronavirus ['Item']
          Professionalisering ['Item']
          Anders organiseren ['Item']
             Coronavirus ['Item']
          Differentiatie ['Item', 'isUserSelectable']
          School en samenleving ['Item', 'isUserSelectable']
          Het volgen van leerlingen ['Item', 'isUserSelectable']
          21ste Eeuwse vaardigheden ['Item']
       Stickers ['System']
          SlimFit onderwijs ['User generated tags', 'Item']
          Voortgezet Leren []
          Slimmerlerenmetict.nl -> PO Raad ['Item']
          Schoolleiders voor de Toekomst ['Item']
          Kennisnet podcasts ['Item']
          Lesopafstand.nl ['System', 'Item']
          Meesterwerk Podcast ['System', 'Item']
          test-sticker ['System', 'Item']
          School aan Zet ['System', 'Item']
          Leraren me

In [12]:
# handig.find_out_of_sync_parent_child_relationships(Tag)


In [13]:
name = 'leiden'

In [14]:
%%sql
 -- # VERMOEDELIJK NIET NODIG ; de search query hierboven met de CTE levert alle tags al 
 -- # op die bij een search opgelepeld moeten worden 
 -- # explain (analyze,costs, buffers, verbose)
select 
    tag.name, 
    concat_ws('/',t0.search, t1.search, t2.search, t3.search, t4.search, t5.search, t6.search),
    array_remove(ARRAY[t0.search, t1.search, t2.search, t3.search, t4.search, t5.search, t6.search],null)
  from tag  
 inner join mv__tag_arrays t0 on t0.id = tag.id 
 left outer join mv__tag_arrays t1 on t1.gid = ANY(t0.children)
 left outer join mv__tag_arrays t2 on t2.gid = ANY(t1.children)
 left outer join mv__tag_arrays t3 on t3.gid = ANY(t2.children)
 left outer join mv__tag_arrays t4 on t4.gid = ANY(t3.children)
 left outer join mv__tag_arrays t5 on t5.gid = ANY(t4.children)
 left outer join mv__tag_arrays t6 on t6.gid = ANY(t5.children)
 where tag.name ilike concat('%',:name, '%')
 limit 10

 * postgresql://postgres@pgpool:5432/backend
10 rows affected.


name,concat_ws,array_remove
begeleiden van leerlingen bij profielwerkstuk,begeleiden van leerlingen bij profielwerkstuk,['begeleiden van leerlingen bij profielwerkstuk']
Leiden,Leiden,['Leiden']
AED Leiden,AED Leiden,['AED Leiden']
Leiden Kennisstad,Leiden Kennisstad,['Leiden Kennisstad']
Het Vlietland College Scholengemeenschap voor Vwo Havo Mavo LEIDEN,Het Vlietland College Scholengemeenschap voor Vwo Havo Mavo LEIDEN,['Het Vlietland College Scholengemeenschap voor Vwo Havo Mavo LEIDEN']
Driestar College Leiden LEIDEN,Driestar College Leiden LEIDEN,['Driestar College Leiden LEIDEN']
Het Waterland LEIDEN,Het Waterland LEIDEN,['Het Waterland LEIDEN']
Stedelijk Gymnasium Athena LEIDEN,Stedelijk Gymnasium Athena LEIDEN,['Stedelijk Gymnasium Athena LEIDEN']
Stedelijk Gymnasium Socrates LEIDEN,Stedelijk Gymnasium Socrates LEIDEN,['Stedelijk Gymnasium Socrates LEIDEN']
Leonardo College LEIDEN,Leonardo College LEIDEN,['Leonardo College LEIDEN']


## item-search
Item search is bedoeld om snel te kunnen zoeken door een platte tabel te maken met zoveel mogelijk gegevens die gebruikt worden voor het doorzoeken van items. 
De kolommen moeten het filteren mogelijk maken om in te zoeken. Tags en subtags zijn 2 kolommen, omschrijving, auteur, enz .

In [158]:
%%sql
create unique index if not exists idx_item__gid__id on public.item (gid) include (id); 
create unique index if not exists idx_tag__gid__id on public.tag (gid) include (id); 
commit

 * postgresql://postgres@pgpool:5432/backend
Done.
Done.
Done.


[]

In [19]:
%%sql
--# TODO: toevoegen aan migrate 
CREATE EXTENSION unaccent;

 * postgresql://postgres@pgpool:5432/backend
Done.


[]

In [20]:
%%sql
select lower(unaccent(normalize('𝔰𝔬𝔪𝔢𝔫𝔞𝔪𝔢ĕĒÖ',nfkc)))

 * postgresql://postgres@pgpool:5432/backend
1 rows affected.


lower
somenameeeo


In [78]:
%%sql
drop materialized view if exists mv__itemsearch; 
create materialized view mv__itemsearch as 
--# explain (analyze,costs, buffers, verbose)  
  select 
    item.gid, 
    lower(unaccent(normalize(string_agg(tag_children.name,';'),nfkc)))  as tags,
    lower(unaccent(normalize(item.name, nfkc))) as item_name, 
    lower(unaccent(normalize(item.short_description, nfkc))) as item_short_description, 
    lower(unaccent(normalize("user".email,nfkc))) as user_email,  
    lower(unaccent(normalize("user".name, nfkc))) as user_name, 
    "user".email not like '%@roc.nl' as publicly_visible,
    to_tsvector('dutch', 
                lower(unaccent(normalize(string_agg(tag_children.name,';'),nfkc))) || ' ' ||
                lower(unaccent(normalize(item.name, nfkc))) || ' ' ||
                lower(unaccent(normalize(item.short_description, nfkc)))||' '|| 
                lower(unaccent(normalize("user".name, nfkc)))) as vectors
  from item
    inner join tag on tag.gid = ANY(array_remove(regexp_split_to_array(item.tags, E'\\|'), ''))
    inner join "user" on "user".gid = item.author 
    , fn_recursive_parent_tags(tag.gid) as tag_children --# geen clause want dat zit al in het argument?!
 where item.platform = 'SvS'
    --and item.gid = '347b1a2f-aef1-43a2-b2eb-735d3643e6a8' 
    group by item.gid, item.name, item.short_description, "user".email, "user".name 

 * postgresql://postgres@pgpool:5432/backend
Done.
891 rows affected.


[]

In [132]:
%%sql
select * 
from mv__itemsearch 
--where publicly_visible=False
  where vectors @@ to_tsquery('dutch','mindful &!symbool')

 * postgresql://postgres@pgpool:5432/backend
0 rows affected.


gid,tags,item_name,item_short_description,user_email,user_name,publicly_visible,vectors


In [61]:
commit

 * postgresql://postgres@pgpool:5432/backend
Done.


[]

In [62]:
%%sql
select * 
  from mv__itemsearch 
 where gid = 'wereldorienatie' 
    or item_name like '%wereldorientatie%' 
    or item_short_description like '%wereldorientatie%'
    or tags like '%wereldorientatie%'
    or user_name like '%wereldorientatie%'
    or user_email like '%wereldorientatie%' 

 * postgresql://postgres@pgpool:5432/backend
6 rows affected.


[('0beef933-2c2d-446b-8a5e-72eeb97c2dc7', "lezen;user generated tags;uirelated;system;onderzoekend leren;user generated tags;uirelated;system;leerinhoud;dimensie;organisatiebrede ontwikkeling; ... (459 characters truncated) ... lated;system;primair onderwijs (po);sector;leesonderwijs;user generated tags;uirelated;system;onderwijskwaliteit;user generated tags;uirelated;system", 'doelgericht leren lezen', "op [basisschool 't heem](https://hetheemnijverdal.nl/) in nijverdal wordt begrijpend lezen en technisch lezen samengevoegd tot een  leermoment, waarb ... (765 characters truncated) ... op lerarenontwikkelfonds.nl lees je [meer over deze praktijk](https://www.lerarenontwikkelfonds.nl/initiatief/lezen-om-te-leren-leren-om-te-lezen/).*", 'eddie@educationwarehouse.nl', 'eddie ', True),
 ('2bc8aa4a-d823-42d5-86c4-fa7d8cbb7b3d', 'daltononderwijs;user generated tags;uirelated;system;personeel;dimensie;onderzoekend leren;user generated tags;uirelated;system;leerinhoud;dimensie;l ... (273 characters truncated) ... 0 t/m 12 jaar (bovenbouw po);doelgroep;leraar, pedagogisch medewerker of coach;geplaatst door;groepsontwikkeling;omvang;primair onderwijs (po);sector', 'onderzoeksvragen formuleren in het basisonderwijs', 'op basisschool [de meene](https://www.gelderveste.nl/scholen/daltonschool-de-meene/contact) in zelhem wordt extra aandacht besteed aan het formuleren ... (1341 characters truncated) ... ontwikkelfonds.nl lees je [meer over deze praktijk.](https://www.lerarenontwikkelfonds.nl/initiatief/onderzoeksvragen-formuleren-op-de-basisschool/)*', 'eddie@educationwarehouse.nl', 'eddie ', True),
 ('313b6b35-d1c0-41ac-836b-3ab7adad3645', 'leerlijn ontwikkelen;user generated tags;uirelated;system;personeel;dimensie;thematisch onderwijs;user generated tags;uirelated;system;leerinhoud;dim ... (330 characters truncated) ... oelgroep;leraar, pedagogisch medewerker of coach;geplaatst door;primair onderwijs (po);sector;onderwijskwaliteit;user generated tags;uirelated;system', "'mijn bewijsdoelen'", "bij kindcentrum [talent.nl]( https://kindcentrumtalent.nl/) wordt gewerkt aan een (digitaal) overzicht van doelen in een meerjarenplan die aan nieuw  ... (1266 characters truncated) ... 2'. \r\n\r\n*op lerarenontwikkelfonds.nl lees je [meer over deze praktijk.] \r\n(https://www.lerarenontwikkelfonds.nl/initiatief/mijn-bewijsdoelen/)*", 'eddie@educationwarehouse.nl', 'eddie ', True),
 ('89d9ae56-ce82-419b-aaff-6bfa3713d4e6', 'autonomie;user generated tags;uirelated;system;motivatie;user generated tags;uirelated;system;lezen;user generated tags;uirelated;system;leerinhoud;d ... (359 characters truncated) ... oach;geplaatst door;lof2018;user generated tags;uirelated;system;primair onderwijs (po);sector;met document(en);media;schoolbrede ontwikkeling;omvang', 'door lezen de wereld ontmoeten', 'op [basisschool hulsberg](https://www.bs-hulsberg.nl/) in hulsberg wordt sinds 2018 gewerkt aan het anders organiseren van begrijpend leesonderwijs,  ... (5420 characters truncated) ... r\n*op lerarenontwikkelfonds.nl lees je [meer over deze praktijk](https://www.lerarenontwikkelfonds.nl/initiatief/door-lezen-de-wereld-ontmoeten-2/)*', 'linda.vaessen@innovo.nl', 'linda vaessen', True),
 ('95978b2f-68db-414f-90e3-3c6cd9dd5d87', 'slimfit onderwijs;stickers;user generated tags;uirelated;uirelated;system;system;thematisch onderwijs;user generated tags;uirelated;system;leerinhoud ... (424 characters truncated) ... stem;primair onderwijs (po);sector;met document(en);media;schoolbrede ontwikkeling;omvang;vakoverstijgend werken;user generated tags;uirelated;system', 'thematisch werken', "op [de masten](https://www.kcmeerlaer.nl/) in rosmalen wordt er voor wereldorientatie en de creatieve vakken thematisch gewerkt. het team besluit aan ... (237 characters truncated) ... r een themaplanning en een weekplanning. bekijk hieronder een aantal voorbeelden, of maak gebruik van het lege format om zelf een thema te ontwerpen!", 'directie@kcmeerlaer.nl', 'anita van heusden', True),


## Leidse items: item counts per gebruikte tag

In [165]:
%%sql
create index if not exists idx_mv__item_tags__item_gid on public.mv__item_tags (item_gid) include (tag_gid);
create index if not exists idx_mv__item_tags__tag_gid on public.mv__item_tags (tag_gid) include (item_gid);
commit

 * postgresql://postgres@pgpool:5432/backend
Done.
Done.
Done.


[]

In [3]:
%%sql
explain (analyze, costs, buffers, verbose)

with leidse_items as (
select item_gid
  from mv__item_tags where tag_gid ='1274163a-be86-4395-8eef-4af66d9b81ec'
)
select mv__item_tags.tag_gid, mv__item_tags.tag_name, count(distinct leidse_items.item_gid) 
  from leidse_items 
    inner join mv__item_tags on leidse_items.item_gid = mv__item_tags.item_gid 
 group by mv__item_tags.tag_gid, mv__item_tags.tag_name 

 * postgresql://postgres@pgpool:5432/backend
29 rows affected.


QUERY PLAN
GroupAggregate (cost=500.50..506.24 rows=287 width=45) (actual time=10.509..10.891 rows=107 loops=1)
"Output: mv__item_tags.tag_gid, mv__item_tags.tag_name, count(DISTINCT mv__item_tags_1.item_gid)"
"Group Key: mv__item_tags.tag_gid, mv__item_tags.tag_name"
Buffers: shared hit=7 read=202
-> Sort (cost=500.50..501.22 rows=287 width=53) (actual time=10.477..10.502 rows=244 loops=1)
"Output: mv__item_tags.tag_gid, mv__item_tags.tag_name, mv__item_tags_1.item_gid"
"Sort Key: mv__item_tags.tag_gid, mv__item_tags.tag_name"
Sort Method: quicksort Memory: 53kB
Buffers: shared hit=7 read=202
-> Hash Join (cost=480.28..488.78 rows=287 width=53) (actual time=10.205..10.295 rows=244 loops=1)



<hr size=6>

# Optimalisatie van Postgres
## Random page cost
https://www.postgresql.org/docs/current/sql-altertablespace.html  
https://wiki.postgresql.org/wiki/Tuning_Your_PostgreSQL_Server

In [40]:
%%sql

alter tablespace pg_global set (random_page_cost = 2)

 * postgresql://postgres@pgpool:5432/backend
Done.


[]

In [41]:
%%sql 
-- create unique index idx_item_tags on public.item (gid) include (tags, array_remove(regexp_split_to_array(tags, E'\\|'), '')); 
-- FeatureNotSupported: expressions are not supported in included columns    
create unique index if not exists idx_item_tags on public.item (gid) include (tags); 



 * postgresql://postgres@pgpool:5432/backend
Done.


[]

In [108]:
%%sql 
commit 

 * postgresql://postgres@pgpool:5432/backend
Done.


[]

## Work_mem
https://www.pgmustard.com/blog/work-mem


In [42]:
%%sql 
show work_mem

 * postgresql://postgres@pgpool:5432/backend
1 rows affected.


work_mem
4MB


In [43]:
%%sql 
-- # TODO: toevoegen aan migrate 
set work_mem to '16MB'; 
select pg_reload_conf();

 * postgresql://postgres@pgpool:5432/backend
Done.
1 rows affected.


pg_reload_conf
True


In [45]:
%%sql 
explain (analyze,costs, buffers, verbose)  
with item_tags as (
    select item.gid, array_remove(regexp_split_to_array(item.tags, E'\\|'), '') as tags
      from item
), used_items as (
    select item.gid, item.name 
      from item inner join "user" on item.author = "user".gid and "user".email not like '%@roc.nl'
)
select item_tags.gid as item_gid, tag.gid as tag_gid   
  from item_tags 
       inner join tag on tag.gid = ANY(item_tags.tags)
       inner join used_items on item_tags.gid = used_items.gid 

 * postgresql://postgres@pgpool:5432/backend
38 rows affected.


QUERY PLAN
Nested Loop (cost=40.82..1658.75 rows=3729 width=74) (actual time=1.037..82.382 rows=12444 loops=1)
"Output: item.gid, tag.gid"
Buffers: shared hit=27029
-> Nested Loop (cost=40.53..428.37 rows=373 width=316) (actual time=0.918..5.639 rows=925 loops=1)
"Output: item.gid, item.tags"
Inner Unique: true
Buffers: shared hit=2056
-> Hash Join (cost=40.25..243.04 rows=373 width=37) (actual time=0.903..2.317 rows=925 loops=1)
Output: item_1.gid
Inner Unique: true


<hr size=15>
# Ontwikkel breek dingen

In [46]:
from pydal import Field 
db.define_table(
    "mv__item_tags",
    Field("item_gid", "uuid"),
    Field("tag_gid", "uuid"),
    Field("item_name", "string"),
    Field("tag_name", "string"),
    migrate=False,  # is a TEMPORARY materialized view
    primarykey=["item_gid"],
    redefine=True 
)

<Table mv__item_tags (item_gid, tag_gid, item_name, tag_name)>

In [47]:
db(db.mv__item_tags).select(limitby=(0,10)).as_list()

[{'item_gid': '000543fd-cfb7-486d-9b7e-620177d6b9a5',
  'tag_gid': '4eba3077-ef20-4044-a44d-2431e45bdacf',
  'item_name': 'Humanics: een vakoverstijgende leerlijn M-stroom',
  'tag_name': 'mens en maatschappij'},
 {'item_gid': '000543fd-cfb7-486d-9b7e-620177d6b9a5',
  'tag_gid': '33cd5a31-cfc9-40d8-a0e2-4ecddf2f85df',
  'item_name': 'Humanics: een vakoverstijgende leerlijn M-stroom',
  'tag_name': 'aardrijkskunde'},
 {'item_gid': '000543fd-cfb7-486d-9b7e-620177d6b9a5',
  'tag_gid': '4225d914-d4c8-4f89-85c5-11e26f853afa',
  'item_name': 'Humanics: een vakoverstijgende leerlijn M-stroom',
  'tag_name': 'Leerinhoud'},
 {'item_gid': '000543fd-cfb7-486d-9b7e-620177d6b9a5',
  'tag_gid': '4df327a3-9484-4f35-bf5a-cfa5b19cdb97',
  'item_name': 'Humanics: een vakoverstijgende leerlijn M-stroom',
  'tag_name': 'Economie'},
 {'item_gid': '000543fd-cfb7-486d-9b7e-620177d6b9a5',
  'tag_gid': '069d0a71-811b-4ef6-98de-147ce35479ba',
  'item_name': 'Humanics: een vakoverstijgende leerlijn M-stroom',
  

In [48]:
db(db.mv__item_tags.item_gid=='000543fd-cfb7-486d-9b7e-620177d6b9a5').select()

<Rows (24)>

In [49]:
db(db.mv__item_tags.tag_gid=='069d0a71-811b-4ef6-98de-147ce35479ba').select()

<Rows (1)>

In [6]:
%%sql
select tag_gid, count(item_gid) 
  from mv__item_tags 
  group by tag_gid 
 having count(item_gid) < 4
 order by count(item_gid) desc 
    limit 1

 * postgresql://postgres@pgpool:5432/backend
1 rows affected.


tag_gid,count
0a8832f7-2ad7-4db8-bfb0-3d126c999cc9,3


In [132]:
from collections import Counter
timings = Counter()
counts = Counter()
from pprint import pp 

In [133]:
import diskcache 

In [134]:
class ItemLoader:
    def keys(self, search:str, with_tags:list, platform:str=None, search_in:list=None):
        found = db(db.item.name.ilike('%'+search+'%')|db.item.short_description.ilike('%'+search+'%')).select(db.item.gid)
        timings['itemloader.keys']+=db._lastsql[-1]
        return len(found), [row[db.item.gid] for row in found]

In [151]:
cache = diskcache.Cache()
@diskcache.memoize_stampede(cache, expire=10)
def get_item_counts_per_tag_for_given_tag_combination_and_search_query(
    q: str, filtered_with: list = None, count_tree=False
):
    select_parameters = {'groupby':db.mv__item_tags.tag_gid}
    # with database_connection() as db:
    # print(filtered_with)
    if True: 
        query = db.mv__item_tags.tag_gid != None
        if filtered_with:
            # with_tags can be a list of tags (uuid)
            # [a,b,c] in which case all are ANDED
            # but when tuples of uuid appear they are ored in the query
            # [(a,b,c), (x,y,z)] ( a or b or c) OR!! (x or y or z)
            # [(a,b,c), d,e] (a or b or c) AND d AND e
            for tag in filtered_with:
                if isinstance(tag, (list, tuple)):
                    tag_list = tag
                    for tag in tag_list:
                        # left outer join voor OR functionaliteit
                        aliased_mv = db.mv__item_tags.with_alias(
                            "mv__item_tags__as__"
                            + tag.replace("-", "")
                            + "_"
                            + str(random.randint(10000, 100000))
                        )
                        select_parameters.setdefault("left", []).append(
                            aliased_mv.on(
                                (aliased_mv.item_gid == db.mv__item_tags.item_gid)
                                & (aliased_mv.tag_gid == tag)
                            )
                        )
                else:
                    # inner join voor AND functionaliteit
                    aliased_mv = db.mv__item_tags.with_alias(
                        "mv__item_tags__as__"
                        + tag.replace("-", "")
                        + "_"
                        + str(random.randint(10000, 100000))
                    )
                    select_parameters.setdefault("join", []).append(
                        aliased_mv.on(
                            (aliased_mv.item_gid == db.mv__item_tags.item_gid)
                            & (aliased_mv.tag_gid == tag)
                        )
                    )

        if q:
            item_count, restrict_to_gids = ItemLoader().keys(
                search=q,
                with_tags=filtered_with,
                platform="SvS",
                search_in=["tag", "name", "description", "author"],
            )
            query &= db.mv__item_tags.item_gid.belongs(restrict_to_gids)
        count = db.mv__item_tags.item_gid.count()
        db_query = db(query)._select(db.mv__item_tags.tag_gid, count, **select_parameters)
        result = db.executesql(db_query)
        timings['get_item_count_for_tags']+=db._lastsql[-1]
        return dict(result)

In [152]:
db.rollback()
result = get_item_counts_per_tag_for_given_tag_combination_and_search_query(q='python', filtered_with=[])
result

{'ed8d8d22-3578-435d-b943-3584fe853792': 1,
 '539632c3-4601-4479-8349-2ba527bbe817': 1,
 'cfebbbba-61ce-41de-b8a8-536e84143363': 1,
 '19478107-c5cb-4e92-8958-92ed9f21cb78': 1,
 '22615965-b307-4d39-b4ea-871e6baa6854': 1,
 '01c86943-5d83-4d2d-9155-81993727794a': 1,
 '7cdfd835-942e-4769-a202-ba9fdbe76bb8': 1,
 '722a1d89-b585-495f-8895-9f12fc0d054a': 1}

In [153]:
def get_item_count_for_tag(
    gid: str, q: str, filtered_with: list = None, count_tree=False
):
    counts['get_item_count_for_tag_hits']+=1
    select_parameters = {}
    # with database_connection() as db:
    print(filtered_with)
    if True: 
        taggid_hits_lookup = get_item_counts_per_tag_for_given_tag_combination_and_search_query(q=q, filtered_with=filtered_with)
        return taggid_hits_lookup.get(gid,0)

                        

In [154]:
%%sql 
select * from mv__item_tags 
where tag_gid = '0a8832f7-2ad7-4db8-bfb0-3d126c999cc9'

 * postgresql://postgres@pgpool:5432/backend
3 rows affected.


item_gid,tag_gid,item_name,tag_name
eec9901c-bfbe-494b-9f8f-c2c45d682bea,0a8832f7-2ad7-4db8-bfb0-3d126c999cc9,Digikeuzebord: 90 jonge kinderen kiezen wat te doen,basisonderwijs
11e743ed-8c36-49c0-b558-6ef55b730838,0a8832f7-2ad7-4db8-bfb0-3d126c999cc9,Verrassende creativiteit door programmeerlessen,basisonderwijs
48bb72af-6e68-4c79-ae6f-5090267d334a,0a8832f7-2ad7-4db8-bfb0-3d126c999cc9,Handelend rekenen tijdens parkeermomenten,basisonderwijs


In [168]:
# parameters
db.rollback()
timings.clear()
counts.clear()
q=''
gid='0a8832f7-2ad7-4db8-bfb0-3d126c999cc9' # deze tag zou 3 items moeten hebben. eec9901c-bfbe-494b-9f8f-c2c45d682bea; 11e743ed-8c36-49c0-b558-6ef55b730838 en 48bb72af-6e68-4c79-ae6f-5090267d334a
filtered_with_should_be_2=['4225d914-d4c8-4f89-85c5-11e26f853afa'] # dit zou 2 items moeten overhouden: 11e743ed-8c36-49c0-b558-6ef55b730838 en 48bb72af-6e68-4c79-ae6f-5090267d334a
filtered_with_should_be_3=[['4225d914-d4c8-4f89-85c5-11e26f853afa','8856b32d-dcfd-4a29-baa7-679888418938']] # dit zou weer de 3 items moeten overhouden
filtered_with_should_be_2_again=[['4225d914-d4c8-4f89-85c5-11e26f853afa','8856b32d-dcfd-4a29-baa7-679888418938'],'4225d914-d4c8-4f89-85c5-11e26f853afa'] # dit zou weer de 2 items moeten overhouden

assert get_item_count_for_tag(gid, q, filtered_with_should_be_2, count_tree=False) == 2, 'platte filter search werkt niet meer'
assert get_item_count_for_tag(gid, q, filtered_with_should_be_3, count_tree=False) == 3, 'ORred lijst wordt niet goed doorzocht'
assert get_item_count_for_tag(gid, q, filtered_with_should_be_2_again, count_tree=False) == 2, 'AND uit de filter overruled de OR binnen de lijst niet. '

q = 'programmeerlessen'
# print(ItemLoader().keys(search=q,with_tags=[]))
print(get_item_count_for_tag(gid, q, count_tree=False))

assert get_item_count_for_tag(gid, q, filtered_with_should_be_2, count_tree=False) == 1, 'platte filter search werkt niet meer met search term'
assert get_item_count_for_tag(gid, q, filtered_with_should_be_3, count_tree=False) == 1, 'ORred lijst wordt niet goed doorzocht met search term'
assert get_item_count_for_tag(gid, q, filtered_with_should_be_2_again, count_tree=False) == 1, 'AND uit de filter overruled de OR binnen de lijst niet, met search term '

pp(timings)
pp(counts)
print('Total:',sum(timings.values()))

['4225d914-d4c8-4f89-85c5-11e26f853afa']
[['4225d914-d4c8-4f89-85c5-11e26f853afa', '8856b32d-dcfd-4a29-baa7-679888418938']]
[['4225d914-d4c8-4f89-85c5-11e26f853afa', '8856b32d-dcfd-4a29-baa7-679888418938'], '4225d914-d4c8-4f89-85c5-11e26f853afa']
None
1
['4225d914-d4c8-4f89-85c5-11e26f853afa']
[['4225d914-d4c8-4f89-85c5-11e26f853afa', '8856b32d-dcfd-4a29-baa7-679888418938']]
[['4225d914-d4c8-4f89-85c5-11e26f853afa', '8856b32d-dcfd-4a29-baa7-679888418938'], '4225d914-d4c8-4f89-85c5-11e26f853afa']
Counter({'itemloader.keys': 0.11013913154602051,
         'get_item_count_for_tags': 0.04596376419067383})
Counter({'get_item_count_for_tag_hits': 7})
Total: 0.15610289573669434


In [144]:
timings.clear()
counts.clear()
# zoek nog door meer tags om een browser view te simuleren
for tag_row in db(db.tag.meta_tags.contains([Tag('user-generated-tags').gid])).select(db.tag.gid):
    get_item_count_for_tag(tag_row[db.tag.gid],q,count_tree=False )

pp(timings)
pp(counts)
print('Total:',sum(timings.values()))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [145]:
db(db.tag.meta_tags.contains([Tag('filter').gid])).select(db.tag.gid).as_list()

[{'gid': 'efd63edb-753f-49e0-85b5-07cf1e304ec2'},
 {'gid': '0d1f6695-2e79-457d-9696-37b6d67dfe5b'},
 {'gid': '9d1ed795-afc2-41e1-bbe4-36a063750cfb'},
 {'gid': '9cecefbe-e997-44d4-b572-6701e3a21da5'},
 {'gid': 'b2179e3e-12b0-433f-a7b6-be14e759302d'},
 {'gid': 'cdfcacb5-bf1b-4878-af5d-514c8033f61e'},
 {'gid': 'b2cd0060-2ec4-483e-831a-17a28afdeacc'}]

In [197]:
db._lastsql

('SELECT "tag"."gid" FROM "tag" WHERE ("tag"."meta_tags" ILIKE \'%|8b78e33d-c4ab-442e-aaf6-411fda089b03|%\' ESCAPE \'\\\');',
 0.02033400535583496)

In [52]:
#db.item(gid='11e743ed-8c36-49c0-b558-6ef55b730838')

In [51]:
# %%sql 
# select * 
#  from mv__item_tags root
#     inner join item on item.gid::uuid = root.item_gid 
#     --inner  join mv__item_tags t1 on t1.item_gid = root.item_gid and t1.tag_gid = '4225d914-d4c8-4f89-85c5-11e26f853afa'        
#  where root.tag_gid = '0a8832f7-2ad7-4db8-bfb0-3d126c999cc9' 
# limit 10 

#  Bijwerken materialized views
https://taiga.edwh.nl/project/remco-ewcore/task/232

> Dirty markeren van tabellen via een trigger:
> ```sql
CREATE TRIGGER log_update
AFTER UPDATE OR INSERT OR DELETE ON item
FOR EACH ROW
WHEN (OLD.* IS DISTINCT FROM NEW.*)
EXECUTE FUNCTION mark_materialized_view_as_dirty('item');

> De `log_table_changed(tablename)` functie moet dan een vlaggetje aangeven in een `dirty_materialized_views` tabel zodat een getimede functie kan zien of een update ook echt nodig is.
Hiermee wordt voorkomen dat we elke 5 minuten een zware bewerking uitvoeren, of dat we lang moeten wachten om een refesh uit te voeren. De opschoonfunctie verwijderd dan de regels uit de `dirty_materialized_views`.

In [28]:
%%sql
-- # TODO: toevoegen aan migrate.py
-- # create table dirty_materialized_views 
create table if not exists dirty_materialized_views
(
    id                     serial 
        constraint dirty_materialized_views_pk primary key,
    materialized_view_name varchar(128)            not null,
    ts                     timestamp default now() not null,
    source_description     varchar(256)
);

comment on column dirty_materialized_views.source_description is 'possibly a desription, tablename or triggername. For traceability. ';

create unique index if not exists dirty_materialized_views_id_uindex
    on dirty_materialized_views (id);
commit; 

 * postgresql://postgres@pgpool:5432/backend
Done.
Done.
Done.
Done.


[]

In [53]:
%%sql
--# add triggers to mark alterations to the table 
CREATE or REPLACE FUNCTION mark_materialized_view_as_dirty() returns trigger AS $$

    DECLARE
        view_name varchar(128);
        description varchar(256); 
    BEGIN 
        view_name := TG_ARGV[0];
        description := concat(TG_ARGV[1],':',old.id,'/',new.id);
        insert into dirty_materialized_views (materialized_view_name, source_description) values (view_name, description);
        RETURN new;
    END;
$$ LANGUAGE plpgsql;
commit;

 * postgresql://postgres@pgpool:5432/backend
Done.
Done.


[]

create triggers on table `mark_dirty__{table}{[counter]}__{materialized view name}`

In [61]:
def new_dirty_maker(mv_name, table):
    db.executesql(f'''
    CREATE OR REPLACE TRIGGER mark_dirty__{table}__{mv_name}
        AFTER UPDATE OR INSERT OR DELETE ON item
        FOR EACH ROW
    EXECUTE function mark_materialized_view_as_dirty ('{mv_name}', '{table}');
    ''')
    print(db._lastsql[0])
    db.commit()

In [64]:
new_dirty_maker('mv__item_tags','tag')
new_dirty_maker('mv__item_tags','item')


    CREATE OR REPLACE TRIGGER mark_dirty__tag__mv__item_tags
        AFTER UPDATE OR INSERT OR DELETE ON item
        FOR EACH ROW
    EXECUTE function mark_materialized_view_as_dirty ('mv__item_tags', 'tag');
    


In [ ]:
%%sql 
-- python code om dirty_materialized_views uit te lezen en te verwerken
-- deze code in microservice zetten 